In [6]:
"""
TAKE HOME TEST FOR DATA SCIENCE ROLE

**Dependencies**
Python libraries requires to run script
    `pandas` 

**Set up**
Please make sure the following files are located in your working directory before running script
    `fullbas.csv`
    `simplebas.csv`
    `orgcard.csv`
"""
### SET UP
import pandas as pd
import os

# Use this section to navigate to appropriate directory - it is assumed that CSV data is in your Downloads folder
os.chdir("Downloads")  
print(os.getcwd())  

### IMPORT DATA
full_bas = pd.read_csv("fullbas.csv")
simple_bas = pd.read_csv("simplebas.csv")
org_card = pd.read_csv("orgcard.csv")

### SEE DATA QUALITIES
# Check out var names and data format
#full_bas.head()
#simple_bas.head()
#org_card.head()

### FIRST OBSERVATIONS
# Both full and simple BAS look to be one record per report run, though the format is definitely different
# User ID & Org ID look like they are in the same format so should match up.
# Dates are in different Formats so will need to convert. Min and Max dates of each data set overlap showing transitionary change over
# Am assuming dates are both in UTC
# Full BAS report name is a label while Simple BAS report identifier is a code (for now I will need to assume that somewhere has info to convert report code to appropriate label)
# PROBLEM: Simple BAS has no OrgID so cannot be joined to ORGCARD - do we have a users table (with orgID) we could use as an intermediary join?? - This means we can't filter to only active and paying orgs for both full and simple.

min_fullbas_date = full_bas['datestring'].min()
max_fullbas_date = full_bas['datestring'].max()
min_simbas_date = simple_bas['datetime'].min()
max_simbas_date = simple_bas['datetime'].max()

# There is only one level to Report Name in Full BAS but 963 in simple - this seems unusual, though there are different versions. The questions dont ask to breakdown by report so I will assume it's fine to accept this structure
distinct_fullbas_reports = full_bas['reportname'].nunique()
distinct_simbas_reports = simple_bas['shortcode'].nunique()

### MAKING A SINGLE EASY TO USE DATASET
# Data is conceptually the same, but formatted differently so in order to be able to work with simple BAS and full BAS as a whole they'll need to be added together
full_bas['datetime'] = pd.to_datetime(full_bas['datestring'] + ' ' + full_bas['timestring'])


# Bring only user ID and Timestamp (as that's all that's needed to answer questions - with the exception of orgid but we can't join that to simple bas)
simplebas_subset = simple_bas[['userid', 'datetime']]
fullbas_subset = full_bas[['userid', 'datetime']]

# Create source field to identify which table the run comes from
simplebas_subset['Source'] = 'Simple'
fullbas_subset['Source'] = 'Full'

# Stach the two source datasets to one workable dataset
combined_bas = pd.concat([simplebas_subset, fullbas_subset], ignore_index=True)

### LOOKING FOR PATTERNS & TRENDS
# Make Datetime all the same format (was different for each source)
combined_bas['datetime'] = pd.to_datetime(combined_bas['datetime'])
# Check no null values in Dates as I was having trouple aggregating by month to start with
#print(combined_bas['datetime'].isnull().sum())

# Was getting an error about mized time zones - am assuming both UTC, but in real life may been to be converted to UTC first
combined_bas['datetime'] = pd.to_datetime(combined_bas['datetime'], utc=True)

# Create a month column & Aggregate for answer to Q1
combined_bas['month_year'] = combined_bas['datetime'].dt.strftime('%Y-%m')
monthly_count = combined_bas.groupby('month_year').size().reset_index(name='row_count')

simple_bas_users = simple_bas['userid'].unique()
full_bas_users = full_bas['userid'].unique()
common_users = set(simple_bas_users) & set(full_bas_users)
num_common_users = len(common_users)

### answers to questions
# ● What month saw the most report runs? - 2018-08 with 12598 (this is not limited to active and paying orgs as per orgcard/simple bas join issue with data)
# ● How many orgs ran both of the reports? - we can't find this unless we find a way to join orgcard to simplebas (perhaps check if a user table exists?). If we had this we would do something similiar to the SQL secion one problem.
# ● How many users have used either of the reports? - 1470 users are present in both simple and full bas reports

### The following shouldn't really be done without having org card joined to simple_bas, though for the sake of answering the questions I am now only looking at full BAS (and active/paying orgs as this info is now available to all orgs)
# ● How many users have run reports for multiple organisations? - 277 users have run full BAS only reports for multiple orgs
# ● Which pricing plans are the most popular for orgs using BAS? - Premium 5 with 49333, again this is only Full BAS

paying_org_card = org_card[(org_card['organisationstatus'] == 'Active') & (org_card['payingflag'] == 1)]

# # Org card ID is uppercase while full bas is upper & lower (This is an actual problem with some XERO data)
full_bas['orgid'] = full_bas['orgid'].str.upper()
full_bas_paying = pd.merge(full_bas, paying_org_card, left_on='orgid', right_on='organisationid', how='inner')

users_multiple_orgs = full_bas_paying.groupby('userid')['orgid'].nunique()
users_multiple_orgs = users_multiple_orgs[users_multiple_orgs > 1]
#print(users_multiple_orgs.count())

pricing_plan_counts = full_bas_paying['productoption'].value_counts()
print(pricing_plan_counts)

C:\Users\SophieFlentge\Downloads


C:\Users\SophieFlentge\AppData\Local\Temp\ipykernel_17776\2768854969.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  simplebas_subset['Source'] = 'Simple'
C:\Users\SophieFlentge\AppData\Local\Temp\ipykernel_17776\2768854969.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fullbas_subset['Source'] = 'Full'
C:\Users\SophieFlentge\AppData\Local\Temp\ipykernel_17776\2768854969.py:68: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=Tr

productoption
Premium 5                           49333
Standard                            45278
Premium 20                          37625
Premium                             37318
Premium 50                          21804
Premium 100                         10349
GST Cashbook                         7333
Starter                              4975
Premium 20 + Expenses                1709
Premium 5 + Expenses                 1518
Premium 5 + Projects                 1300
Payroll Cashbook                     1283
Premium 50 + Expenses                1197
Ledger                               1190
Standard + Expenses                  1028
Standard + Projects                   519
Premium 20 + Projects                 467
Premium 100 + Projects                444
Premium + Expenses                    437
Premium 100 + Expenses                422
Premium 5 + Projects + Expenses       269
Non-GST Cashbook                      252
Premium + Projects + Expenses         221
Premium 50 + Project